In [4]:
import pandas as pd
import numpy as np
import sys
import os

# Add project root to path
sys.path.append(r"D:\data-problem-solving")

from db import load_table, engine

projects = pd.read_sql('SELECT * FROM "risky_projects_linkedin".linkedin_projects', engine)
emp_projects = pd.read_sql('SELECT * FROM "risky_projects_linkedin".linkedin_emp_projects', engine)
employees = pd.read_sql('SELECT * FROM "risky_projects_linkedin".linkedin_employees', engine)

In [6]:
# --- JOIN TABLES ---
df = (
    projects
    .merge(emp_projects, left_on='id', right_on='project_id', how='inner')
    .merge(employees, left_on='emp_id', right_on='id', how='inner', suffixes=('_proj', '_emp'))
)

# --- ENSURE DATE COLUMNS ARE DATETIME ---
df['start_date'] = pd.to_datetime(df['start_date'])
df['end_date'] = pd.to_datetime(df['end_date'])

# --- CALCULATE PROJECT DURATION (in years) ---
df['duration_years'] = (df['end_date'] - df['start_date']).dt.days / 365.0

# --- CALCULATE TOTAL EXPENSES PER PROJECT (VECTORIZED VERSION, no apply warning) ---
df['prorated_cost'] = df['salary'] * df['duration_years']

agg_df = (
    df.groupby(['id_proj', 'title', 'budget'], as_index=False, observed=True)
      .agg(total_expenses=('prorated_cost', lambda x: np.ceil(x.sum())))
)

# --- FILTER PROJECTS WHERE EXPENSES > BUDGET ---
risky_projects = agg_df[agg_df['total_expenses'] > agg_df['budget']]

# --- SORT BY TOTAL_EXPENSES DESC ---
risky_projects = risky_projects.sort_values('total_expenses', ascending=False)

# --- FINAL OUTPUT ---
risky_projects = risky_projects.rename(columns={'title': 'project_name'})[
    ['project_name', 'budget', 'total_expenses']
]

print(risky_projects)

   project_name   budget  total_expenses
25    Project26  36190.0         79368.0
23    Project24  11918.0         74665.0
31    Project32  12356.0         66523.0
5      Project6  41611.0         63230.0
11    Project12  10468.0         62843.0
36    Project37   8806.0         61949.0
24    Project25  38909.0         57975.0
20    Project21  24330.0         57310.0
19    Project20  19497.0         55962.0
29    Project30  24011.0         53106.0
1      Project2  32487.0         52870.0
32    Project33  30110.0         50034.0
43    Project44  22885.0         49271.0
28    Project29  10935.0         48371.0
17    Project18  10302.0         46381.0
8      Project9  32341.0         44691.0
45    Project46   9824.0         42314.0
13    Project14  30014.0         36774.0
0      Project1  29498.0         36293.0
10    Project11  11705.0         31606.0
3      Project4  15776.0         30656.0
34    Project35  23931.0         28652.0
41    Project42  24934.0         28301.0
35    Project36 